# vina_demo

## Colab Setup

1. Check the runtime instance
2. Choose whether to use Google Drive (persistence) or temp `/content`
3. Install required Python packages
4. Get course files/data into the runtime
5. Mount Google Drive for persistent storage


### 1) Check the runtime


In [8]:
import sys, platform, os, textwrap
print("Python:", sys.version.split()[0])
print("Platform:", platform.platform())
print("Working dir:", os.getcwd())

!arch

Python: 3.12.12
Platform: Linux-6.6.105+-x86_64-with-glibc2.35
Working dir: /content
x86_64


## 2) Choose whether to use Google Drive (persistence)

- If you want your edits to persist across sessions, use **Drive**.
- If you're just running a quick exercise, you can skip Drive and use the temporary Colab filesystem (`/content`).


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
import os
from pathlib import Path

# Change this folder name once; everything else uses it.
COURSE_DIR_NAME = "structbio_course"  # you can rename for your course

USE_DRIVE = True

if "drive" in str(Path("/content/drive")) and os.path.exists("/content/drive") and USE_DRIVE:
    ROOT = Path("/content/drive/MyDrive") / COURSE_DIR_NAME
else:
    ROOT = Path("/content") / COURSE_DIR_NAME

ROOT.mkdir(parents=True, exist_ok=True)

DATA_DIR = ROOT / "data"
(DATA_DIR).mkdir(exist_ok=True)

OUTPUTS_DIR = ROOT / "outputs"
(OUTPUTS_DIR).mkdir(exist_ok=True)

print("ROOT:", ROOT)
print("data:", DATA_DIR)
print("outputs:", OUTPUTS_DIR)


ROOT: /content/drive/MyDrive/structbio_course
data: /content/drive/MyDrive/structbio_course/data
outputs: /content/drive/MyDrive/structbio_course/outputs


### 2) Copy over the input data files and code needed for the vina demo

... and unzip the archive


In [21]:
import os, pathlib, urllib.request

RAW_URL = "https://raw.githubusercontent.com/vvoelz/chem5412-spring2026/main/data/assignment02.zip"
zip_path = pathlib.Path(DATA_DIR) / "assignment02.zip"

urllib.request.urlretrieve(RAW_URL, zip_path)
print("Saved to:", zip_path)

!unzip {zip_path}
!cp -r assignment02 {DATA_DIR}

print(f'Unzipped folder `assignment02` moved to {DATA_DIR}')

Saved to: /content/drive/MyDrive/structbio_course/data/assignment02.zip
Archive:  /content/drive/MyDrive/structbio_course/data/assignment02.zip
replace assignment02/input_files/4ey7.pdb? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: assignment02/input_files/4ey7.pdb  
  inflating: assignment02/input_files/human_AChE_binders.csv  
  inflating: assignment02/input_files/4ey7_receptor_prepped.pdb  
  inflating: assignment02/input_files/rivastigmine/rivastigmine_out.pdbqt  
  inflating: assignment02/input_files/rivastigmine/rivastigmine.pdbqt  
  inflating: assignment02/input_files/rivastigmine/rivastigmine.mol2  
  inflating: assignment02/input_files/rivastigmine/config_singledock  
  inflating: assignment02/input_files/rivastigmine/rivastigmine.smiles  
  inflating: assignment02/input_files/huperzine/huperzine.smiles  
  inflating: assignment02/input_files/huperzine/huperzine.pdbqt  
  inflating: assignment02/input_files/huperzine/huperzine.mol2  
  inflating: assignment02/input_fi